In [17]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd

driver = webdriver.Chrome(ChromeDriverManager().install())

tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


In [18]:
driver.get("https://www.google.com.br/")

produto = 'iphone 12 64gb'

driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

elementos = driver.find_elements(By.CLASS_NAME, 'hdtb-mitem')

for elemento in elementos:
    if "Shopping" in elemento.text:
        elemento.click()
        break

lista_precos = driver.find_elements(By.CLASS_NAME, 'XrAfOe')
lista_nomes = driver.find_elements(By.CLASS_NAME, 'EI11Pd')
lista_links = driver.find_elements(By.CLASS_NAME, 'eaGTj ')

for elemento_preco in lista_precos:
    preco = elemento_preco.text
    print(preco)






R$ 3.623,00
R$ 3.623,00.
R$ 4.184,07
R$ 4.184,07.
R$ 3.500,00
R$ 3.500,00.
Usado
R$ 3.300,00
R$ 3.300,00.
R$ 3.329,10
R$ 3.329,10.
Recondicionado
R$ 3.300,00
R$ 3.300,00.
R$ 3.199,00
R$ 3.199,00.
R$ 4.699,00 R$ 7.999,00
R$ 4.699,00 Custava R$ 7.999,00.
R$ 3.099,00
R$ 3.099,00.
R$ 4.899,00
R$ 4.899,00.
R$ 4.604,65
R$ 4.604,65.
R$ 3.300,00
R$ 3.300,00.
R$ 4.049,10
R$ 4.049,10.
R$ 2.969,10
R$ 2.969,10.
Recondicionado
R$ 2.739,00
R$ 2.739,00.
Recondicionado
R$ 4.249,00
R$ 4.249,00.
R$ 3.300,00
R$ 3.300,00.
R$ 4.684,06
R$ 4.684,06.
R$ 3.336,56
R$ 3.336,56.
(€ 589,00)
R$ 4.436,24
R$ 4.436,24.
R$ 4.999,00
R$ 4.999,00.
R$ 4.249,00
R$ 4.249,00.
R$ 3.700,00
R$ 3.700,00.
R$ 2.997,00
R$ 2.997,00.
R$ 4.049,10
R$ 4.049,10.
R$ 4.089,90
R$ 4.089,90.
R$ 2.529,00
R$ 2.529,00.
R$ 2.146,95
R$ 2.146,95.
(€ 379,00)
R$ 4.249,00
R$ 4.249,00.
R$ 2.719,00
R$ 2.719,00.
Recondicionado
R$ 5.799,00
R$ 5.799,00.
R$ 2.350,00
R$ 2.350,00.
Usado
R$ 4.199,00
R$ 4.199,00.
R$ 4.949,10 R$ 5.499,00
R$ 4.949,10 Custava R$ 5